# Density Estimation and Normalising Flows

In science, we often times deal with complicated highly dimensional probability distributions. This motivates the need to extract meaningful information from them using a flexible approach. This is where we can use normalising flows (NF), a class of invertible generative model. NF can learn the probability distribution over random variable $X$ from a set of observed data $x$ with probability density $p_\theta(x)$ parameterized with a neural network.

1. Density estimation, where we can evaluate likelihood of new points under the model
2. Sample new data, where we can identify correlation structure in very high-dimensional datasets and obtain the conditional probability $p(x|y)$ of some property x given y.

Before going into NF, we briefly put them into context by discussing Maximum Likelihood Estimation (MLE). We then introduce normalising flows that expand this analysis to the non-parametric regime. We then demonstrate the performance of NFs on the moon data-set. An exercise to use it for real-data is shown at the end.

In [ ]:
%matplotlib inline

from IPython.display import Image

import torch
from torch import nn
from torch import distributions
from torch.nn.parameter import Parameter

from sklearn import cluster, datasets, mixture
from sklearn.preprocessing import StandardScaler

import numpy as np
import math

import matplotlib
import matplotlib.pyplot as plt

# we will read in data with pandas frame
import pandas as pd

import seaborn as sns

from cycler import cycler
from IPython.display import Image

Let us start by first considering the familiar, straightforward 1D case of a Gaussian distribution.

In [ ]:
# create an Gaussian distribution instance
mu_truth = 3. # mean
sigma_truth = 5. # standard deviation
normal_distribution = torch.distributions.normal.Normal(loc=mu_truth, scale=sigma_truth);

# draw from the distribution
x_torch = normal_distribution.sample(sample_shape=torch.Size([10000]))
plt.hist(x_torch.numpy(), bins=100, range=[-25,25], density=True, color='grey');

# compare that with the analytic p(x) expression from the same instance
x_range = torch.linspace(-25,25,100)
y_prob = torch.exp(normal_distribution.log_prob(x_range))
plt.plot(x_range, y_prob, color='b', lw=5.0);

plt.xlabel("x")
plt.ylabel("p(x)")


We often only observe the realisations $\{x_i\}$. What we want to characterise, however, is the distribution, $p(x)$, from which the $\{x_i\}$ were generated. In other words, what is the generative process leading to the observed values?

The classical way of performing this task is through MLE. If we assume a parameteric form of the distribution, i.e., $p(x) = p(x|\theta)$, where $\theta$ is the parameter of the distribution, then the best distribution that describe the data $\{x_i\}$ would be the one that maximizes the $p(x_i)$, or more precisely,

$$\hat{\theta} = \underset{\theta}{\mathrm{argmax}} \prod_i p(x_i|\theta) $$

Take the example above. If we know that the data is drawn from a Gaussian distribution, but we do not know what the mean and variance of the Gaussian distribution, MLE states that the Gaussian distribution which describes the data the best would be the one with $\theta = \{\mu, \theta\}$.

In [ ]:
# draw from the distribution
plt.hist(x_torch.numpy(), bins=100, range=[-25,25], density=True);

# the correct distribution should "envelope" the data
x_range = torch.linspace(-25,25,100)
y_prob = torch.exp(normal_distribution.log_prob(x_range))
plt.plot(x_range, y_prob, lw=5, label="maximizes the likelihood");

# if we use the wrong mu and sigma, the likelihood will be worse
normal_wrong = torch.distributions.normal.Normal(loc=mu_truth+10., scale=sigma_truth);
y_prob = torch.exp(normal_wrong.log_prob(x_range))
plt.plot(x_range, y_prob, color="black", ls="--");

normal_wrong = torch.distributions.normal.Normal(loc=mu_truth, scale=sigma_truth*3.);
y_prob = torch.exp(normal_wrong.log_prob(x_range))
plt.plot(x_range, y_prob, color="black", ls="--", label="worse likelihood");

plt.legend(frameon=False)

plt.ylim([0,0.12])
plt.xlabel("x")
plt.ylabel("p(x)")

It is oftern more numerically stable to take the logarithm and the negative component, such that we end up minimizing the negative log-likelihood:

$$\hat{\theta} = \underset{\theta}{\mathrm{argmin}}-\sum_i \log p(x_i|\theta)$$

If we assume $p(x)$ follows a Gaussian distribution, we have $\theta = (\mu, \sigma)$ and

$$ p(x_i | \mu, \sigma) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp \left( \frac{(x_i-\mu)^2}{2\sigma^2} \right) $$

$$ \log p(x_i | \mu, \sigma) = -0.5 \log \big( 2 \pi \sigma^2 \big) -  \frac{(x_i-\mu)^2}{2\sigma^2} $$

With pytorch, this will transfer into:

In [ ]:
# it is funny that pytorch does not have a pre-defined pi
sigma = nn.Parameter(torch.rand(1, requires_grad=True))
mu = nn.Parameter(torch.rand(1, requires_grad=True))
lr = 0.01
params = [sigma, mu]
optimizer = torch.optim.Adam(params, lr=lr)

torch_pi = torch.tensor(math.pi)

def loss_fn(sample, sigma, mu):
    log_prob = -0.5*torch.log(2*torch_pi*sigma**2) \
                - (sample-mu)**2/(2*sigma**2)
    return -torch.mean(log_prob)

num_epochs = 20000

for e in range(num_epochs):
    
    loss = loss_fn(x_torch, sigma, mu)
    optimizer.zero_grad() 
    loss.backward() 
    optimizer.step()
        
    if e % 1000 == 0:
        print('iter %s:' % e, 'training loss = %.5f' % loss)
    
print("Best estimated sigma:", sigma.detach().numpy()[0])
print("Best estimate mean:", mu.detach().numpy()[0])

print(" ")
print("True sigma:", sigma_truth)
print("True mean:", mu_truth)

## Normalising Flows

However, in the case of MLE example shown above, we have access to the parametric form of the likelihood.

The goal of a NF model is to find a transformation, $f$, such that a multi-modal, complicated, often times many-dimensional $p(x)$ distribution can be transformed into, and hence expressed by, a more tractable "base" distribution $p(z)$. A normal distribution is often chosen to be the base distribution, which earns the methodology the name.

In other words, NFs perform a change of variables mapping:

$$ f: x \mapsto z, $$
where $p(z)$ is Gaussian.

We can also think of it as a function $g$ molding the base distribution into the more complicated p(x):

$$ g: z \mapsto x, $$

with $z = f(x) = g^{-1}(z)$.

Let's start with straighforward 1D Gaussian example 


In [ ]:
normal_distribution = torch.distributions.normal.Normal(loc=0, scale=1);
z_torch = normal_distribution.sample(sample_shape=torch.Size([10**5]))
plt.hist(z_torch.numpy(), bins=100, range=[-4,4], density=True);

plt.xlabel("z")
plt.ylabel("p(z)")

Now let's assume that our x is a sigmoid function of $2z$.
$$ x \equiv g(z) = \frac{e^{2z}}{e^{2z}+1} $$

In [ ]:
x_torch = torch.sigmoid(2*z_torch)
plt.hist(x_torch.numpy(), bins=100, density=True);

plt.xlabel("x")
plt.ylabel("p(x)") #looking groovy

Recall that the inverse function of a Sigmoid function is the logit function, i.e., in our case,

$$ z = g^{-1}(x) = \frac{1}{2} \log \frac{x}{1-x} $$

Therefore, if we perform a clever change of random variables: $x \mapsto z.$ 

In [ ]:
x_inverse = torch.logit(x_torch)/2.
plt.scatter(x_torch, x_inverse)
plt.xlabel("x")
plt.ylabel(r"$\mathregular{z = g^{-1}(x)}$")

In [ ]:
plt.hist(x_inverse.numpy(), bins=100, density=True);
plt.xlabel("z")
plt.ylabel("p(z)") #we recover the p(z)

How about the moons dataset? it would be tricky to come up with an analytical function to transform this to a 2D Gaussian.

In [ ]:
x = datasets.make_moons(n_samples=1000, noise=.05)[0].astype(np.float32) #p(x)
plt.scatter(x[:, 0], x[:, 1], c='m')
plt.title(r'$X \sim p(X)$')

Intuitively, we can think of the transformation $f$ as warping the space D in order to mold the density $p(z)$ into $p(x)$. The probability mass in dx must equal the probability mass in dz. So, if dz is expanded, then the density at x is smaller than the density at z and viceversa. The absolute Jacobian determinant quantifies the relative change of volume of a small neighbourhood around z due to f.

At their heart, normalising flows are neural-network based transformations that perform a change of variable. Unlike other neural networks, for normalising flow to serve as an effective way to perform the change of random variable and to learn non-trivial probability distribution functions, we need to effect two main conditions. 

1. we require that our model is **invertible** such that we can generate new data points efficiently by sampling from the base distribution and inverting the results. 

2. we need expressive enough functions $f$, which we refer to as transforms, capable of recovering the intricacies of our input data space and that have a **tractable and efficiently computable Jacobian determinant.** 

The question now becomes how to design the transforms that satisfy the two criteria. There have been many proposed architectures such as [Glow](https://arxiv.org/abs/1807.03039), [Neural Spline Flows](https://arxiv.org/abs/1906.04032), and the list keep growing every year. One good one to start comes from the Real-valued Non-Volume Preserving [RealNVP](https://arxiv.org/abs/1605.08803).

## Introducing RealNVP

Let $x$ to be a $D$-dimensional variables in the original variable space. The transformation freezes a subset of $d$-dimensional variables. Without loss of generality, let's assume these $d$ dimensions to be the first $d$ dimensions. The other $D-d$ dimensions are then transformed as:

$$
z_{1:d} = x_{1:d} \qquad\qquad(1)
$$

$$ z_{d+1:D} = x_{d+1:D} \odot \exp(s(x_{1:d})) + t(x_{1:d}) \qquad\qquad (2)$$

All the subscripts here designate the subset of dimensions we consider for each transformation. $\odot$ denotes the element-wise product.

The first equation states that we are only performing an identity mapping which maps the first $d$ dimensional variables to themselves. The second equation is slightly more involved. As a starter, we have two MLP networks $s$ and $t$ which maps a $\mathbb{R}^d$ input to a $\mathbb{R}^{D-d}$. 

Now let's see if these transformations obey the two conditions.

**Condition 1**: it is easily invertible.

$$
z_{1:d} = x_{1:d} 
$$


$$ x_{d+1:D} = ( z_{d+1:D} - t(z_{1:d}) ) \odot \exp ( -s(z_{1:d}) ) $$
 
**Condition 2**: easily computable Jacobian.

Knowing that:
$$
z_{1:d} = x_{1:d} 
$$

$$ z_{d+1:D} = x_{d+1:D} \odot \exp(s(x_{1:d})) + t(x_{1:d})$$

Differentiating individual $z_i$ with respect to $x_i$ and then taking the determinant will yield the Jacobian.

$$
J =
\left|
\begin{matrix}
\mathbb{1}_{d\times d} & \mathbb{0}_{d \times (D-d)} \\
\frac{\partial z_{d+1:D}}{\partial x_{1:d}} & {\rm diag}(\exp (s(x_{1:d}))
\end{matrix}
\right|.
$$

$$det(J) = \prod_{j=d+1}^{D} \exp (s(x_{1:d}))_j.$$

Since computing the inverse does not require computing the inverse of neither the two transformations, $s$ and $t$, and computing the Jacobian determinant does not involve computing their Jacobian either, we can model these two functions as being arbitrarily complex, which is why we can choose them to be deep neural networks.

In [ ]:
class MLP(nn.Module):
    def __init__(self, num_inputs, num_hiddens):
        super(MLP, self).__init__()
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(num_inputs, num_hiddens),
            torch.nn.ReLU(), 
            torch.nn.Linear(num_hiddens, num_hiddens),
            torch.nn.ReLU(),
            torch.nn.Linear(num_hiddens, num_inputs),
        )
        
    def forward(self, x):
        return self.mlp(x)
    
    
#-------------------------------------------------------------------  
num_inputs = 5
num_hiddens = 64

# initiate the two MLPs, s and t
s_network = MLP(num_inputs, num_hiddens)
t_network = MLP(num_inputs, num_hiddens)

**Training RealNVP**

Our objective function is to ask how well the ensemble of transformed $\{ f(x_i) \}$ follows a Gaussian distribution. The goal is to optimize the parameters $\theta$ (in RealNVP, the parameters in the MLP networks $s$ and $t$), such that:

$$\hat{\theta} = \underset{\theta}{\mathrm{argmin}} -\sum_i \log p_z(f(x_i)) \cdot J_{x_i}$$

$$\hat{\theta} = \underset{\theta}{\mathrm{argmin}} -\sum_i \log p_z(f(x_i)) - \sum_i \log J_{x_i}$$

$$\hat{\theta} = \underset{\theta}{\mathrm{argmin}} -\sum_i \log p_z(f(x_i)) - \sum_i \sum_{j=d+1}^D s(x_{i,1:d})_j$$

When evaluating the loss function, it suffices to return the log of the determinant, which is nothing but the sum of all the components in $s(x_{1:d})$. 

Now let's implement RealNVP. Note we define two networks, sig_net and mu_net and we implement it following the math:

In [ ]:
class RealNVP(nn.Module):
    def __init__(self, D, d, prior, hidden):
        super().__init__()
        self.D, self.d = D, d
        self.prior = prior
        self.sig_net = nn.Sequential(
                    nn.Linear(d, hidden),
                    nn.LeakyReLU(),
                    nn.Linear(hidden, D - d))

        self.mu_net = nn.Sequential(
                    nn.Linear(d, hidden),
                    nn.LeakyReLU(),
                    nn.Linear(hidden, D - d))

    def forward(self, x):
        x1, x2 = x[:, :self.d], x[:, self.d:] 
        sig = self.sig_net(x1)
        z1, z2 = x1, x2 * torch.exp(sig) + self.mu_net(x1)
    
        z_hat = torch.cat([z1, z2], dim=-1)

        log_jacob = sig.sum(-1)
        
        return z_hat, log_jacob
    
    def inverse(self, Z):
        z1, z2 = Z[:, :self.d], Z[:, self.d:] 
        
        x1 = z1
        x2 = (z2 - self.mu_net(z1)) * torch.exp(-self.sig_net(z1))
 
        return torch.cat([x1, x2], -1)
    
    def log_prob(self,x):
        z, logp = self.forward(x)
        
        return self.prior.log_prob(z) + logp


In [ ]:
D = 2
d = 1
prior = distributions.MultivariateNormal(torch.zeros(2), torch.eye(2))
model = RealNVP(D, d, prior, hidden=512)

In [ ]:
optimizer = torch.optim.Adam([p for p in model.parameters() if p.requires_grad==True], lr=1e-3)

for t in range(5001):
    # generate moons at each epoch
    noisy_moons = datasets.make_moons(n_samples=100, noise=.05)[0].astype(np.float32)
    loss = -model.log_prob(torch.from_numpy(noisy_moons)).mean()
    
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()
    
    if t % 500 == 0:
        print('iter %s:' % t, 'loss = %.3f' % loss)

In [ ]:
#let's make some toy data!
noisy_moons = datasets.make_moons(n_samples=1000, noise=.05)[0].astype(np.float32)

def plot_noisy_moons(model):

    z = np.random.multivariate_normal(np.zeros(2), np.eye(2), 1000)
    plt.subplot(222)
    plt.scatter(z[:, 0], z[:, 1]) #p(z)
    plt.title(r'$z \sim p(z)$')

    plt.subplot(221)
    x = datasets.make_moons(n_samples=1000, noise=.05)[0].astype(np.float32) #p(x)
    plt.scatter(x[:, 0], x[:, 1], c='r')
    plt.title(r'$X \sim p(X)$')
    
    z = model.forward(torch.from_numpy(noisy_moons))[0].detach().numpy() #let's see how our functions are transforming 
    #the complicated data
    plt.subplot(223)
    plt.scatter(z[:, 0], z[:, 1])
    plt.title(r'$z = f(X)$')
    
plot_noisy_moons(model)
#we notice that our transformation hasn't done wonders in transforming the moons to the prior
#however some change is happening
#what if we have the model become a normalising "flow", i.e. more coupled transforms?

We notice that one single unit of an RealNVP doesn't recreate the target distribution. Since a invertible and differentiable function, i.e. a difeomorphism, is closed under composure, we can easily add more units to see if we recover the distribution.

![flow](https://lilianweng.github.io/posts/2018-10-13-flow-models/normalizing-flow.png)

In [ ]:
#more compact code using PyTorch RealNVP tutorial
#We achieve this by randomly masking some of the dimensions of the input at each pass.

class RealNVP(nn.Module):
    def __init__(self, nets, nett, mask, prior):
        super(RealNVP, self).__init__()
        
        self.prior = prior
        self.mask = nn.Parameter(mask, requires_grad=False)
        self.t = torch.nn.ModuleList([nett() for _ in range(len(masks))]) #the same as before but now we have 
                                                                          #multiple MLPs to deal with each composable transform
        self.s = torch.nn.ModuleList([nets() for _ in range(len(masks))])
        
    def inverse(self, z):
        x = z
        for i in range(len(self.t)):
            x_ = x*self.mask[i]
            s = self.s[i](x_)*(1 - self.mask[i])
            t = self.t[i](x_)*(1 - self.mask[i])
            x = x_ + (1 - self.mask[i]) * (x * torch.exp(s) + t)
        return x

    def forward(self, x):
        log_det_J, z = x.new_zeros(x.shape[0]), x
        for i in reversed(range(len(self.t))):
            z_ = self.mask[i] * z
            s = self.s[i](z_) * (1-self.mask[i])
            t = self.t[i](z_) * (1-self.mask[i])
            z = (1 - self.mask[i]) * (z - t) * torch.exp(-s) + z_
            log_det_J -= s.sum(dim=1)
        return z, log_det_J
    
    def log_prob(self,x):
        z, logp = self.forward(x)
        return self.prior.log_prob(z) + logp
        
    def sample(self, batchSize): 
        z = self.prior.sample((batchSize, 1))
        logp = self.prior.log_prob(z)
        x = self.inverse(z)
        return x

In [ ]:
#slight reformulation of the net
nets = lambda: nn.Sequential(nn.Linear(2, 256), nn.LeakyReLU(), nn.Linear(256, 256), nn.LeakyReLU(), nn.Linear(256, 2), nn.Tanh())
nett = lambda: nn.Sequential(nn.Linear(2, 256), nn.LeakyReLU(), nn.Linear(256, 256), nn.LeakyReLU(), nn.Linear(256, 2))

masks = torch.from_numpy(np.array([[0, 1], [1, 0]] * 3).astype(np.float32))
prior = distributions.MultivariateNormal(torch.zeros(2), torch.eye(2))
flow = RealNVP(nets, nett, masks, prior)

In [ ]:
#the training routine
optimizer = torch.optim.Adam([p for p in flow.parameters() if p.requires_grad==True], lr=1e-3)

for t in range(2001):    
    noisy_moons = datasets.make_moons(n_samples=1000, noise=.05)[0].astype(np.float32)
    loss = -flow.log_prob(torch.from_numpy(noisy_moons)).mean()
    
    optimizer.zero_grad() 
    loss.backward(retain_graph=True)
    optimizer.step()
    
    if t % 500 == 0:
        print('iter %s:' % t, 'loss = %.3f' % loss)

In [ ]:
plot_noisy_moons(flow)

In [ ]:
# we can also sample from the model
z_sample = prior.sample(sample_shape=torch.Size([500]))

# perform the inverse function to get x
x_test = flow.inverse(z_sample)

plt.scatter(x_test[:, 0].detach().numpy(), x_test[:, 1].detach().numpy())

## Application to real data

Now let's go back to science after playing with the toy dataset.

> The Data

In [ ]:
# Each participant is encouraged to use their own dataset, but for the purpose 
# I will showcase the result for APOGEE data
df = pd.read_csv('https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/files/vault/sfabbro/apogee.csv')
df.describe()

In [ ]:

x_torch = torch.tensor(df.values, dtype=torch.float32)
print(x_torch.shape)

# dimension
num_inputs = x_torch.shape[1]

In [ ]:
#for completeness we put the model here again

class RealNVP(nn.Module):
    def __init__(self, nets, nett, mask, prior):
        super(RealNVP, self).__init__()
        
        self.prior = prior
        self.mask = nn.Parameter(mask, requires_grad=False)
        self.t = torch.nn.ModuleList([nett() for _ in range(len(masks))]) #the same as before but now we have 
                                                                          #multiple MLPs to deal with each composable transform
        self.s = torch.nn.ModuleList([nets() for _ in range(len(masks))])
        
    def inverse(self, z):
        x = z
        for i in range(len(self.t)):
            x_ = x*self.mask[i]
            s = self.s[i](x_)*(1 - self.mask[i])
            t = self.t[i](x_)*(1 - self.mask[i])
            x = x_ + (1 - self.mask[i]) * (x * torch.exp(s) + t)
        return x

    def forward(self, x):
        log_det_J, z = x.new_zeros(x.shape[0]), x
        for i in reversed(range(len(self.t))):
            z_ = self.mask[i] * z
            s = self.s[i](z_) * (1-self.mask[i])
            t = self.t[i](z_) * (1-self.mask[i])
            z = (1 - self.mask[i]) * (z - t) * torch.exp(-s) + z_
            log_det_J -= s.sum(dim=1)
        return z, log_det_J
    
    def log_prob(self,x):
        z, logp = self.forward(x)
        return self.prior.log_prob(z) + logp
        
    def sample(self, batchSize): 
        z = self.prior.sample((batchSize, 1))
        logp = self.prior.log_prob(z)
        x = self.inverse(z)
        return x

In [ ]:
# defining the model

num_hidens = 64
nets = lambda: nn.Sequential(nn.Linear(num_inputs, num_hidens), nn.LeakyReLU(), nn.Linear(num_hidens, num_hidens), nn.LeakyReLU(), nn.Linear(num_hidens, num_inputs), nn.Tanh())
nett = lambda: nn.Sequential(nn.Linear(num_inputs, num_hidens), nn.LeakyReLU(), nn.Linear(num_hidens, num_hidens), nn.LeakyReLU(), nn.Linear(num_hidens, num_inputs))

# define mask
num_blocks = 8
masks = []
for i in range(num_blocks):
    mask_layer = np.random.randint(2,size=(num_inputs))
    masks.append(mask_layer)
masks = torch.from_numpy(np.array(masks).astype(np.float32))

prior = distributions.MultivariateNormal(torch.zeros(num_inputs), torch.eye(num_inputs))
flow = RealNVP(nets, nett, masks, prior)
# number of parameters in this flow
print("We are training a deep learning model with ",\
      sum(p.numel() for p in flow.parameters()),
      " parameters")

> Training routine

In [ ]:
num_epochs = 100
batch_size = 512

nsamples = x_torch.shape[0]
nbatches = nsamples // batch_size

# we will optimize all parameters in the 2 x n_blocks s and t MLP networks
optimizer = torch.optim.Adam([p for p in flow.parameters() if p.requires_grad==True], lr=1e-4)

for e in range(num_epochs):

    # randomly permute the data
    perm = torch.randperm(nsamples)

    for i in range(nbatches):
        idx = perm[i * batch_size : (i+1) * batch_size]
        loss = -flow.log_prob(x_torch[idx]).mean()
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

    if e % 10 == 0:
        print('iter %s:' % e, 'loss = %.3f' % loss)



Let's check the results on the distributions of Mg and Fe for stars

In [ ]:
# draw from p(z)
# we draw the exact number of sample as our training data to have a fairer comparison
z_torch = prior.sample(sample_shape=torch.Size([x_torch.shape[0]]))

# perform the inverse function to get p(x)
x_test = flow.inverse(z_torch)

fig = plt.figure(figsize=(20,10)) 
plt.subplot(221)
plt.hexbin(x_test[:,3].detach().numpy(),\
           x_test[:,0].detach().numpy()-x_test[:,3].detach().numpy(), bins=50,\
           cmap="gist_rainbow", label="Estimated")
plt.title(r'Samples')
plt.xlim(-1.5, 1)
plt.ylim(-0.25,0.25)
plt.ylabel('[Mg/Fe]')
plt.xlabel('[Fe/H]')

plt.subplot(222)
plt.hexbin(x_torch[:,3].detach().numpy(),\
           x_torch[:,0].detach().numpy()-x_torch[:,3].detach().numpy(), bins=50,\
           cmap="gist_rainbow", label="Estimated")
plt.title(r'Truth')
plt.xlim(-1.5,1)
plt.ylim(-0.25,0.25)
plt.xlabel('[Fe/H]')